In [13]:
from urllib.request import urlopen
import urllib
import pandas as pd
from bs4 import BeautifulSoup
from enum import Enum
import re
import requests
from PyPDF2 import PdfMerger
from urllib.parse import urlparse
from io import BytesIO

In [2]:
init_url = "https://www.nature.com/articles/s41560-024-01529-3"

In [3]:
dois = []
docs = []
# hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
#          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
#          'Referer': 'https://cssspritegenerator.com',
#          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
#          'Accept-Encoding': 'none',
#          'Accept-Language': 'en-US,en;q=0.8',
#          'Connection': 'keep-alive'}

In [17]:
def get_page(url):
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:106.0) Gecko/20100101 Firefox/106.0')
    req.add_header('Accept', 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8')
    req.add_header('Accept-Language', 'en-US,en;q=0.5')

    r = urllib.request.urlopen(req).read().decode('utf-8')
    return r

In [4]:
def get_base_url(url):
    parsed = urlparse(url)
    return f"{parsed.scheme}://{parsed.netloc}"


In [5]:
def get_paper_from_scholar(url):
    return None # TODO - implement way to get google scholar papers (right now HTTP 403 error)
    if 'scholar.google' not in url:
        return None
    page = urlopen(url)
    html_content = page.read().decode("utf-8")
    soup = BeautifulSoup(html_content, "html.parser")
    parent = soup.find("div", class_="gs_r gs_or gs_scl")
    link = parent.find("a", href=True)
    return link["href"] if link else None
    


In [6]:
def get_text(soup):
    for script in soup(['script', 'style']):
        script.extract()
    text = soup.get_text(separator=' ')
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    return text

In [15]:
def download_all_pdfs(url, paper_index):
    page = get_page(url)
    soup = BeautifulSoup(page.read().decode("utf-8"), 'html.parser')
    pdf_links = [link.get('href') for link in soup.find_all('a', href=True) \
                if link.get('href') and link.get('href').endswith('.pdf')]
    pdf_links = list(set(pdf_links))
    pdf_files = []
    i = 0
    merger = PdfMerger()
    for pdf_link in pdf_links:
        pdf_url = pdf_link if pdf_link.startswith('http') else \
            get_base_url(url) + pdf_link
        res = requests.get(pdf_url)
        merger.append(BytesIO(res.content))
        # pdf_name = f"data/pdfs/{paper_index}_{i}.pdf"
        # with open(pdf_name, 'wb') as f:
        #     f.write(res.content)
        # pdf_files.append(pdf_name)
        i += 1
    # for pdf in pdf_files:
    #     merger.append(pdf)
    with open(f'data/pdfs/{paper_index}.pdf', 'wb') as merged:
        merger.write(merged)
    merger.close()




In [8]:
def get_dois(url, max_depth, curr_depth=1):
    if curr_depth == max_depth:
        return []
    try:
        page = urlopen(url)
        html_content = page.read().decode("utf-8")
        soup = BeautifulSoup(html_content, "html.parser")
    except:
        return []
    a_tags = soup.find_all('a', href=True)
    doi_pattern = r'https:\/\/doi.org\/10.+'
    for a in a_tags:
        match = re.search(doi_pattern, a['href'], re.IGNORECASE)
        if match:
            new_url = match.group()
        else:
            new_url = a['href'] if 'sciencedirect' in a['href'] \
                else get_paper_from_scholar(a['href'])
        if new_url and new_url not in dois:
            dois.append(new_url)
            docs.append(get_text(soup))
            get_dois(new_url, max_depth, curr_depth+1)
    return dois

In [9]:
get_dois(init_url, 2)

['https://doi.org/10.1038/s41560-024-01623-6',
 'https://doi.org/10.1126%2Fscience.adm9474',
 'https://doi.org/10.1038%2Fs41566-019-0398-2',
 'https://doi.org/10.1038%2Fs41560-020-00749-7',
 'https://doi.org/10.1126%2Fscience.abq7652',
 'https://doi.org/10.1038%2Fs41467-021-23616-9',
 'https://doi.org/10.1016%2Fj.matt.2023.05.028',
 'https://doi.org/10.1021%2Fjacs.5b11740',
 'https://doi.org/10.1021%2Facs.chemrev.0c01006',
 'https://doi.org/10.1126%2Fscience.abd4860',
 'https://doi.org/10.1021%2Facsenergylett.6b00327',
 'https://doi.org/10.1038%2Fs41586-022-04604-5',
 'https://doi.org/10.1021%2Facsaem.3c00101',
 'https://doi.org/10.1002%2Faenm.201803699',
 'https://doi.org/10.1002%2Faenm.202202438',
 'https://doi.org/10.1126%2Fsciadv.abj7930',
 'https://doi.org/10.1038%2Fs41467-022-30426-0',
 'https://doi.org/10.1039%2FD0TA10535J',
 'https://doi.org/10.1002%2Fadma.202006238',
 'https://doi.org/10.1002%2Faenm.202201490',
 'https://doi.org/10.1126%2Fscience.adi4107',
 'https://doi.org/10

In [10]:
len(dois)

150

In [18]:
download_all_pdfs("https://www.science.org/doi/10.1126/science.adm9474",10)

HTTPError: HTTP Error 403: Forbidden

In [22]:
for i, doi in enumerate(dois):
    download_all_pdfs(doi, i)

/articles/s41560-024-01623-6.pdf
/articles/s41560-024-01623-6.pdf
/articles/s41560-024-01623-6.pdf


PdfReadError: EOF marker not found

In [104]:
df = pd.DataFrame({
    'id': [i for i in range(1, len(dois) + 1)],
    'url': dois,
    'text': docs
})
df.head()

,id,url,text
0,1,https://doi.org/10.1038/s41560-024-01623-6,Molecular cation and low-dimensional perovskit...
1,2,https://doi.org/10.1038/s41560-024-01529-3,Author Correction: Molecular cation and low-di...
2,3,https://doi.org/10.1126%2Fscience.adm9474,Molecular cation and low-dimensional perovskit...
3,4,https://doi.org/10.1038%2Fs41566-019-0398-2,Molecular cation and low-dimensional perovskit...
4,5,https://doi.org/10.1038/s41566-019-0462-y,Surface passivation of perovskite film for eff...


In [106]:
docs[0]

'Molecular cation and low-dimensional perovskite surface passivation in perovskite solar cells | Nature Energy\nSkip to main content\nThank you for visiting nature.com. You are using a browser version with limited support for CSS. To obtain\nthe best experience, we recommend you use a more up to date browser (or turn off compatibility mode in\nInternet Explorer). In the meantime, to ensure continued support, we are displaying the site without styles\nand JavaScript.\nAdvertisement\nView all journals\nSearch\nLog in\nExplore\ncontent\nAbout\nthe journal\nPublish\nwith us\nSign up for alerts\nRSS feed\nnature\nnature energy\nreview articles\narticle\nMolecular cation and low-dimensional perovskite surface passivation in perovskite solar cells\nDownload PDF\nReview Article\nPublished:\n04 July 2024\nMolecular cation and low-dimensional perovskite surface passivation in perovskite solar cells\nSam Teale\nORCID:\norcid.org/0000-0001-9638-3453 1 \xa0 na1 \xa0 nAff4 ,\nMatteo Degani\nORCID:\n

In [ ]:
def download_pdf(url, index):
    response = requests.get(url) 
    pdf = open(f"paper_pdfs/{index}.pdf", "wb")
    pdf.write(response.read())
    pdf.close()

In [ ]:
index = 0
for doi in dois:
    download_pdf(doi, index)
    index += 1